# Text Generation using LSTM Network
#### ディープラーニングによる文章生成

## What is LSTM?
  
テキスト生成は一種の言語モデリング問題。  
言語モデリングは、テキスト読み上げ、会話システム、テキスト要約など、多数の自然言語処理タスクの中心的な問題。    
訓練された言語モデルは、テキストで使用されている前の一連の単語に基づいて、単語の出現の可能性を学習する。  
言語モデルは、文字レベル、n-gramレベル、文レベル、または段落レベルでも操作できる。  
このノートでは、最先端のリカレントニューラルネットワークを実装してトレーニングすることによって、自然言語テキストを生成するための言語モデルを作成する方法について説明していく
。

## 今回はニュースの本文からタイトルを自動生成します  
  
#### Process  
1. データの準備  
2. 文章のお掃除（記号削除、小文字統一）  
3. 単語に切り分ける  
4. トークン化＝数値化  
5. パディングで変数の長さを統一  
6. LSTMの実装

## 1. Import the libraries ライブラリのインポート

In [8]:
import pandas as pd
import numpy as np

## 2. Load the dataset データのロード

In [9]:
import os
path = os.getcwd()
print(path)

/Users/akr712/Desktop/トレーニング待ち/文章生成


In [17]:
ls

0315_Text_Generation_with_GRU.ipynb  Text_Generation_with_GRU.ipynb
New York Times/


In [18]:
headlines = []
for filename in os.listdir(path):
    if "Articles" in filename:
        article_df = pd.read_csv(path + "/New York Times/" + filename)
        headlines.extend(list(article_df["headline"].values))
        break
        
headlines = [ h for h in headlines if h != "Unknown" ]
print("The number of headline is :", len(headlines))

The number of headline is : 0


In [16]:
headlines

[]

## 3. Dataset preparation 前処理

### 3.1 Dataset cleaning  データクリーニング
  
記号を取り除き、「文字」と「数字」だけ残す。また小文字で統一する。

In [6]:
import string
import warnings
warnings.filterwarnings("ignore")
warnings.simplefilter(action='ignore', category=FutureWarning)

def clean_text(headline):
    text = "".join( word for word in headline if word not in string.punctuation ).lower()
    text = text.encode("utf8").decode("ascii", "ignore")
    return text

# 元データのタイトルに含まれる単語群から独自のコーパスを作成
corpus = [ clean_text(headline) for headline in headlines ]

In [7]:
corpus[:5]

[]

### 3.2 Generating Sequence of N-gram Tokens 文章の単語化&数値化

- 自然言語処理では、テキストを単語単位に分解してベクトル化するのが主流である。  
- N-gram は、Morphological Analysis（形態要素解析）に並ぶ代表的な単語の切り出し手法のひとつ。  
- 具体的には、N-gramとは自然言語（テキスト）を連続するN個の文字、もしくはN個の単語単位で切り出す手法のこと。  
- 強みは「コーパス」が事前に入らないこと、弱みは切り出した単語数が肥大化しやすい点。  
- ex:) "I voted for Trump." n=2 => "I voted", "for Trump"

In [ ]:
vocab = []
for line in corpus:
    words = line.split()
    for word in words:
        vocab.append(word)

vocabraly = set(vocab)

In [ ]:
len(vocabraly)

In [ ]:
from keras.preprocessing.text import Tokenizer

tokenizer = Tokenizer(2000)
tokenizer.fit_on_texts(corpus)
word2index = tokenizer.word_index
len(word2index)

In [ ]:
word2index

In [ ]:
dictionary = {}
rev_dictionary = {}
for word, idx in word2index.items():
    if idx > 1406:
        continue
    dictionary[word] = idx
    rev_dictionary[idx] = word

In [ ]:
max(rev_dictionary.keys())

In [ ]:
input_seqences = tokenizer.texts_to_sequences(corpus)

In [ ]:
input_seqences[:5]

In [ ]:
len(input_seqences)

### 3.3 Padding the Sequences and obtain Variables
#### パディングによって固定長データを作り、説明変数を得る

In [ ]:
input_data = []
target = []
for line in input_seqences:
    for i in range(1, len(line)-1):
        input_data.append(line[:i])
        target.append(line[i+1])

In [ ]:
input_data[:5]

In [ ]:
target[:5]

In [ ]:
MAX_LEN = 0
for seq in input_data:
    if len(seq) > MAX_LEN:
        MAX_LEN = len(seq)
MAX_LEN

In [ ]:
input_data = pad_sequences(input_data, maxlen=MAX_LEN, padding="post", truncating="post")

In [ ]:
len(input_data[0])

In [ ]:
input_data.shape

In [ ]:
target = to_categorical(target, num_classes=total_words)

In [ ]:
target

In [ ]:
target.shape

In [ ]:
VOCAB_SIZE = 2001
VOCAB_SIZE

In [ ]:
MAX_LEN

## 4. LSTMs for Text Generation 長短期記憶層アルゴリズムの実装

### 4.1 LSTM ( Long Short-Term Memory  )    
  
1. Input Layer : Takes the sequence of words as input  
2. LSTM Layer : Computes the output using LSTM units. I have added 100 units in the layer, but this number can be fine tuned later.  
3. Dropout Layer : A regularisation layer which randomly turns-off the activations of some neurons in the LSTM layer.  
4. Output Layer : Computes the probability of the best possible next word as output  

![title](https://cdn-images-1.medium.com/max/1600/1*yBXV9o5q7L_CvY7quJt3WQ.png)

In [ ]:
from keras.models import Sequential
from keras.layers import Embedding, GRU, Dense, Dropout
from keras.callbacks import EarlyStopping

In [ ]:
model = Sequential()

"""入力層"""
model.add(Embedding(input_dim=VOCAB_SIZE, output_dim=100, input_length=MAX_LEN))

"""隠れ層"""
model.add(LSTM(units=100))
model.add(Dropout(rate=0.1))

"""出力層 活性化関数は多層のソフトマックス関数"""
model.add(Dense(units=target.shape[1], activation="softmax"))

In [ ]:
"""損失関数と最適化手法の設定"""
model.compile(loss="categorical_crossentropy", optimizer="adam")

In [ ]:
model.summary()

#### LSTMモデルにテストデータを学習させていく！

In [ ]:
"""予測ラベルと正解ラベルを用意する"""
model.fit(input_data, target, batch_size=32, epochs=5, verbose=1)

### 4.2 GRU ( Gated recurrent unit )

In [ ]:
gru_model = Sequential()
gru_model.add(Embedding(input_dim=VOCAB_SIZE, output_dim=100, input_length=MAX_LEN))
gru_model.add(GRU(units=100))
gru_model.add(Dropout(rate=0.1))
gru_model.add(Dense(units=target.shape[1], activation="softmax"))

In [ ]:
gru_model.compile(loss="categorical_crossentropy", optimizer="adam")

In [ ]:
gru_model.summary()

#### GRUモデルにテストデータを学習させていく！

In [ ]:
gru_model.fit(input_data, target, batch_size=32, epochs=5, verbose=1)

## 5. Generating the text タイトルにふさわしいテキストを自動生成する

In [ ]:
from tensorflow import set_random_seed
from numpy.random import seed
set_random_seed(2)
seed(1)

In [ ]:
def text_generater(seed_text, next_words, model, max_sequence_len):
    for _ in range(next_words):
        token_list = tokenizer.texts_to_sequences([seed_text])[0]
        token_list = pad_sequences([token_list], maxlen=max_sequence_len, padding="post")
        predicted = model.predict_classes(token_list, verbose=0)
        
        output_word = ""
        for word,index in tokenizer.word_index.items():
            if index == predicted:
                output_word = word
                break
        seed_text += " "+output_word
    return seed_text.title()

In [ ]:
text1 = "Trump decided"
text_generater(text1, 5, model, MAX_LEN)

In [ ]:
text_generater(text1, 5, gru_model, MAX_LEN)

## I need more data to training I guess.... 学習不足